In [1]:
from IPython.core.display import Image, HTML
from patsy import dmatrices, demo_data, ContrastMatrix, Poly
from sklearn import linear_model, metrics
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import Lasso, LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import collections
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import plotly.plotly as py
import pylab as pl
import seaborn as sns
import sklearn.cross_validation
import statsmodels.api as sm
import string
import warnings

plt.style.use('fivethirtyeight')
%matplotlib inline

## Pre-Task: Describe the goals of your study

To use logistic regression to determine what factors played a part in who survived and who did not in the Titanic sinking.

## Part 1: Aquire the Data

Code didn't work on home mac either. Just pulled from SQL directly.

#### 1. Connect to the remote database

#### 2. Query the database and aggregate the data

In [2]:
# mac path
df = pd.read_csv('/Users/EKandTower/GA-DSI/projects/projects-weekly/project-05/code/train.csv')

#windows path
# df = pd.read_csv('C:/Uers/Elizabeth/GA-DSI/projects/projects-weekly/project-05/code/train.csv')

df.head()

IOError: File /Users/EKandTower/GA-DSI/projects/projects-weekly/project-05/code/train.csv does not exist

#### 5. What are the risks and assumptions of our data? 

## Part 2: Exploratory Data Analysis

#### 1. Describe the Data

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# reading in data with ages populated

# mac path
df1 = pd.read_csv('/Users/EKandTower/GA-DSI/projects/projects-weekly/project-05/code/titanic.csv')

#windows path
# df1 = pd.read_csv('C:/Users/Elizabeth/GA-DSI/projects/projects-weekly/project-05/code/titanic.csv')


#remove index row
df1.pop('index')

# set passenger number as index
df1=df1.set_index('PassengerId')

#impute na with medians
df1.fillna(df1.median(), inplace=True)

df1.head()

In [ ]:
df1.isnull().sum()

In [ ]:
df1.describe()


In [ ]:
df1.info()

In [ ]:
df1.sum()

#### 2. Visualize the Data

In [ ]:
#histograms of numerical data
df1.hist(figsize=(10,12))

In [ ]:
palette = "diverging"
sns.pairplot(df1, kind="scatter", hue="Survived")

## Part 3: Data Wrangling

#### 1. Create Dummy Variables for *Sex* 

In [ ]:
# creating dummies for gender, ticket class, and port of embarkment

gendummy = pd.get_dummies(df1['Sex'], prefix='Sex')
classdummy = pd.get_dummies(df1['Pclass'], prefix='Class')
portdummy = pd.get_dummies(df1['Embarked'], prefix='Port')

In [ ]:
# binning ages

age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 81]

abns = ['5', '15', '25', '35', '45', '55', '65', '75']

ages = pd.cut(df1['Age'], age_bins, labels=abns, right=False)

## Part 4: Logistic Regression and Model Validation

#### 1. Define the variables that we will use in our classification analysis

In [ ]:
orig_colls_to_keep = ['Survived', 'SibSp','Parch']
df2 = df1[orig_colls_to_keep].join(gendummy.ix[:,'Sex_male':])
df2 = df2.join(classdummy)
df2 = df2.join(portdummy)
df2['ages'] = ages.astype(int)
df2.info()

In [ ]:
agedummy = pd.get_dummies(df2['ages'], prefix='Age')
df2 = df2.join(agedummy)
df2.pop('ages')
df2.columns =['Survived','SibSp','Parch','male','1st_Class','2nd_Class','3rd_Class','Port_C','Port_Q','Port_S','Age0-10'\
      ,'Age10-20','Age20-30','Age30-40','Age40-50','Age50-60','Age60-70','Age70-80']
df2.head()

In [ ]:
df2.sort_index(axis=0, inplace=True)

len(df2)

#### 2. Transform "Y" into a 1-Dimensional Array for SciKit-Learn

In [ ]:
s = df2['Survived'].astype(int)
# print s
type(s)

#### 3. Conduct the logistic regression

In [ ]:
train_cols = df2.columns[1:]
logit = sm.Logit(df2['Survived'], df2[train_cols], maxiter=500)
result = logit.fit()

train_cols

#### 4. Examine the coefficients to see our correlations

In [ ]:
def examine_coefficients(model, dftitles):
    dftitles = pd.DataFrame(
        { 'Coefficient' : model.coef_[0] , 'Feature' : dftitles.columns}
    ).sort_values(by='Coefficient')
    return dftitles[dftitles.Coefficient !=0 ]

In [ ]:
model = linear_model.LogisticRegression(penalty = 'l1', C = 10.0) 

X = df2[['SibSp','Parch','male','1st_Class','2nd_Class','3rd_Class','Port_C','Port_Q','Port_S','Age0-10'\
      ,'Age10-20','Age20-30','Age30-40','Age40-50','Age50-60','Age60-70','Age70-80']]
y = df2['Survived']

model.fit(X, y)
datatitle = examine_coefficients(model, X)

In [ ]:
datatitle

In [ ]:
print df2.corr()



In [ ]:
df2.sum()

There were 577 male aboard and 314 females, about 65% of the passengers. Over half of the passengers were in steerage and 72% boarded in Southampton.

#### 6. Test the Model by introducing a *Test* or *Validaton* set 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33)

In [ ]:
print 'Length of X_train: ' + str(len(X_train))
print 'Length of y_train: ' + str(len(y_train))
print
print 'Length of X_test: ' + str(len(X_test))
print 'Length of y_test: ' + str(len(y_test))

In [ ]:
model = linear_model.LogisticRegression(penalty = 'l1', C = 10.0) 
# 
model.fit(X_test, y_test)
datatitle = examine_coefficients(model, X)
datatitle

#### 7. Predict the class labels for the *Test* set

In [ ]:
y_pred = model.predict(X_test)
y_pred

#### 8. Predict the class probabilities for the *Test* set

In [ ]:
y_pp = model.predict_proba(X_test)
y_pp

#### 9. Evaluate the *Test* set

In [ ]:
for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    scores = cross_val_score(model, X, y, scoring=metric)
    print("mean {}: {}, all: {}".format(metric, scores.mean(), scores))

#### 10. Cross validate the test set

In [ ]:

cross_val_score(model, X, y, n_jobs=1, cv=5)

In [ ]:
cross_val_score(model, X, y, n_jobs=1, cv=5).mean()

In [ ]:
logreg_cv = LogisticRegressionCV(Cs=20, solver='liblinear', cv=3, penalty='l1', scoring='f1')
cv_model = logreg_cv.fit(X_train, y_train)

print('best C for class:')
best_C = {logreg_cv.classes_[i]:x for i, (x, c) in enumerate(zip(logreg_cv.Cs_, logreg_cv.classes_))}
print(best_C)

In [ ]:
gs.best_params_

#### 11. Check the Classification Report

In [ ]:
print classification_report(y_test, y_pred)

#### 12. What do the classification metrics tell us?

This is a pretty strong model with precision, recall, and f1-score all averaging 0.84

#### 13. Check the Confusion Matrix

In [ ]:
conmat = np.array(confusion_matrix(y_test, y_pred, labels=[1,0]))

confusion = pd.DataFrame(conmat, index=['long-term', 'short-term'],
                         columns=['predicted_long','predicted_Fatality'])

print(confusion)

#### 14. What does the Confusion Matrix tell us? 

There were 156 true negatives (predicted fatality correctly), 26 false positives/TypeI Error (predicted alive, but actually dead), 28 false negatives/Type-II Error (predicted dead, actually alive), and 85 true positives (perdicted survived correctly).

#### 15. Plot the ROC curve

In [ ]:
logreg = LogisticRegression(random_state=77)
logreg.fit(X_train, y_train)
Y_pred = logreg.predict(X_test)


# generic curve plotting function
def auc_plotting_function(rate1, rate2, rate1_name, rate2_name, curve_name):
    AUC = auc(rate1, rate2)
    # Plot of a ROC curve for class 1 (fatality)
    plt.figure(figsize=[11,9])
    plt.plot(rate1, rate2, label=curve_name + ' (area = %0.2f)' % AUC, linewidth=4)
    plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel(rate1_name, fontsize=18)
    plt.ylabel(rate2_name, fontsize=18)
    plt.title(curve_name + ' fatality', fontsize=18)
    plt.legend(loc="lower right")
    plt.show()

# plot receiving operator characteristic curve
def plot_roc(y_true, y_score):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    auc_plotting_function(fpr, tpr, 'False Positive Rate', 'True Positive Rate', 'ROC')
    
Y_score = logreg.decision_function(X_test)
plot_roc(y_test, Y_score)


#### 16. What does the ROC curve tell us?

The extent to which a model is likely to give true positives (correct readings) versus false positives (incorrect readings). The larger the area under the curve (the space from the lower righthand corner to the ROC curve) the more likely it is that you will give a correct reading.

## Part 5: Gridsearch

#### 1. Use GridSearchCV with logistic regression to search for optimal parameters 

- Use the provided parameter grid. Feel free to add if you like (such as n_jobs).
- Use 5-fold cross-validation.

In [ ]:
logreg = LogisticRegression(solver='liblinear')
C_vals = np.logspace(-5,1,50)
penalties = ['l1','l2']

gs = GridSearchCV(logreg, {'penalty': penalties, 'C': C_vals}, cv=5)
gs.fit(X, y)



#### 2. Print out the best parameters and best score. Are they better than the vanilla logistic regression?

In [ ]:
gs.best_params_

In [ ]:
logreg = LogisticRegression(C=gs.best_params_['C'], penalty=gs.best_params_['penalty'])
cv_model = logreg.fit(X_train, y_train)


cv_pred = cv_model.predict(X_test)

cm = confusion_matrix(y_test, cv_pred, labels=logreg.classes_)

cm = pd.DataFrame(cm, columns=logreg.classes_, index=logreg.classes_)

cm

In [ ]:
print classification_report(y_test, cv_pred, labels=logreg.classes_)

#### 3. Explain the difference between the difference between the L1 (Lasso) and L2 (Ridge) penalties on the model coefficients.

Lasso will find betas that don't have significant impact on the model and drop their coefficients to (or very close to) zero, effectively killing them out of the model (and effectively performing feature elimination). This encourages saparcity.

Ridge doesn't kill weak betas off, it just pushes their coefficients down to the point where they only have token representation in the model.

*smartass response: there is no difference between the difference. They have the same difference.

#### 4. What hypothetical situations are the Ridge and Lasso penalties useful?

Ridge is useful when you have a lot of betas with small coefficients but not if there are a lot of betas with large coefficients. Ridge also needs the betas to be scaled while LASSO is more flexible.

In this project LASSO was a better utility because gender and, to lesser degrees, class and age dominated the model. There were only a couple of fairly small coefficients and Fare was highly correlated with class.

Ridge would be better in a situation where there were many data points that all contributed slightly.

In the home price exercises Ridge would be better because there are so many things that can sway price.

#### 5. [BONUS] Explain how the regularization strength (C) modifies the regression loss function. Why do the Ridge and Lasso penalties have their respective effects on the coefficients?

C is basically 1/alpha so you're talking about the same concept just from the other side of things. It acts to reduce the power of betas' coefficients to either eliminate factors (LASSO) or reduce them so they are more uniform (Ridge)

#### 6.a. [BONUS] You decide that you want to minimize false positives. Use the predicted probabilities from the model to set your threshold for labeling the positive class to need at least 90% confidence. How and why does this affect your confusion matrix?

In [ ]:
y_pp = model.predict_proba(X_test)
y_pp

## Part 6: Gridsearch and kNN

#### 1. Perform Gridsearch for the same classification problem as above, but use KNeighborsClassifier as your estimator

At least have number of neighbors and weights in your parameters dictionary.

In [ ]:
GK_cv = KNeighborsClassifier(n_neighbors=8, weights='distance', n_jobs=5,)
cv_model = GK_cv.fit(X_train, y_train)

#### 2. Print the best parameters and score for the gridsearched kNN model. How does it compare to the logistic regression model?

In [ ]:
gs.best_params_


In [ ]:
print classification_report(y_test, cv_pred, labels=GK_cv.classes_)

classification numbers were 4 points softer than the logistic regression model's.

#### 3. How does the number of neighbors affect the bias-variance tradeoff of your model?

#### [BONUS] Why?

The higher the k the more vairance / less bias you have. As you go further from the point of origin you get a blurier picture.

#### 4. In what hypothetical scenario(s) might you prefer logistic regression over kNN, aside from model performance metrics?

logistic regression would be more useful when there are multiple factors that could impact the outcome. kNN is a very simple method.

#### 5. Fit a new kNN model with the optimal parameters found in gridsearch. 

In [ ]:
def knn_classifier(labeled_points, new_point, k):
    if k % 2 == 0:
        return "Must use odd k, to prevent ties"
        label_distances = []

    for p in labeled_points:

        distance = calculate_distance(p[0],new_point)
        label_distances.append((distance,p[1]))
    
    label_distances.sort() #
  
    prediction = neighbor_vote([label[1] for label in label_distances[:k]])
    return prediction


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=77)

# Split 70/30 into initial data and new points to classify
training_df = pd.DataFrame({'feature1':X_train[:,0],'feature2':X_train[:,1],'class':y_train})
test_df = pd.DataFrame({'feature1':X_test[:,0],'feature2':X_test[:,1],'class':y_test})

#### 6. Construct the confusion matrix for the optimal kNN model. Is it different from the logistic regression model? If so, how?

#### 7. [BONUS] Plot the ROC curves for the optimized logistic regression model and the optimized kNN model on the same plot.

## Part 7: [BONUS] Precision-recall

#### 1. Gridsearch the same parameters for logistic regression but change the scoring function to 'average_precision'

`'average_precision'` will optimize parameters for area under the precision-recall curve instead of for accuracy.

#### 2. Examine the best parameters and score. Are they different than the logistic regression gridsearch in part 5?

#### 3. Create the confusion matrix. Is it different than when you optimized for the accuracy? If so, why would this be?

#### 4. Plot the precision-recall curve. What does this tell us as opposed to the ROC curve?

[See the sklearn plotting example here.](http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html)

## Part 8: [VERY BONUS] Decision trees, ensembles, bagging

#### 1. Gridsearch a decision tree classifier model on the data, searching for optimal depth. Create a new decision tree model with the optimal parameters.

#### 2. Compare the performace of the decision tree model to the logistic regression and kNN models.

#### 3. Plot all three optimized models' ROC curves on the same plot. 

#### 4. Use sklearn's BaggingClassifier with the base estimator your optimized decision tree model. How does the performance compare to the single decision tree classifier?

#### 5. Gridsearch the optimal n_estimators, max_samples, and max_features for the bagging classifier.

#### 6. Create a bagging classifier model with the optimal parameters and compare it's performance to the other two models.